In [1]:
import sys, os 

import numpy as np
import pandas as pd

import scipy as sci
import xarray as xr

# Constants

In [2]:
""" Definitions of growth model constants from Butzin and Pörtner, 2016  """

A_R = 8.660        # Rate of uninhibited growth at reference temperature T_R (% d^-1 g^1/b)
B_R = 0.3055       # Value of allometric exponent at reference temperature Tr
THETA_A = 18145    # Arrhenius temperature (K) for uninhibited reaction kinetics = 17871,85°C
THETA_B = 4258     # Arrhenius temperature (K) = 3984,85°C 
THETA_H =  25234   # Arrhenius temperature (K) for inhibited reaction kinetics = 24960,85°C
T_R = 283          # Reference optimum temperature (K) = 9.85°C
T_H = 286          # Temperature for inhibitive processes (K) = 12.85°C
C_AVG =  0.291     # Independent of temperature and weight constant (% d^-1)

: 

# Setup

In [8]:
input_files = xr.open_mfdataset('gm/input_data/*.nc', decode_times=False)
new_time = pd.date_range('2000-01-01', '2005-01-01', freq='M')
input_files = input_files.assign_coords({'time': new_time})
input_files['thetao']

<xarray.DataArray 'thetao' (time: 60, depth_coord: 21, latitude: 10, longitude: 22)>
dask.array<concatenate, shape=(60, 21, 10, 22), dtype=float32, chunksize=(12, 21, 10, 22), chunktype=numpy.ndarray>
Coordinates:
  * time         (time) datetime64[ns] 2000-01-31 2000-02-29 ... 2004-12-31
  * longitude    (longitude) float64 -11.75 -11.25 -10.75 ... -2.25 -1.75 -1.25
  * latitude     (latitude) float64 47.25 47.75 48.25 ... 50.75 51.25 51.75
  * depth_coord  (depth_coord) float32 0.0 10.0 20.0 30.0 ... 410.0 490.0 580.0
Attributes:
    long_name:   TEMPERATURE
    units:       Celsius_scale
    pointwidth:  1.0
    fnname:      flaggt

In [5]:
input_files['thetao'].transpose('time','latitude','longitude','depth_coord')[0][0][13][0].load()

<xarray.DataArray 'thetao' ()>
array(12.1378145, dtype=float32)
Coordinates:
    time         datetime64[ns] 2000-01-31
    longitude    float64 -5.25
    latitude     float64 47.25
    depth_coord  float32 0.0
Attributes:
    long_name:   TEMPERATURE
    units:       Celsius_scale
    pointwidth:  1.0
    fnname:      flaggt

In [6]:
n=273
m = input_files['thetao'].transpose('latitude','longitude', 'time', 'depth_coord').fillna(-999); m
def doit():
    i = 0
    for lat in range(len(m.latitude)):
        for lon in range(len(m.longitude)):
            for d in range(len(m.depth_coord)):
                if i == n:
                    print(lat,lon,d)
                    return
                i+=1
doit()

0 13 0


In [6]:
m[0][13][0][0].values.ravel()[0]

12.1378145

In [4]:
def saveIt(obj, fname):
    # input_files
    m = obj['thetao'].transpose('latitude','longitude', 'time', 'depth_coord').fillna(-999)
    len_lat, len_lon = len(m.latitude), len(m.longitude)
    len_ts, len_depth = len(m.time), len(m.depth_coord)

    output = np.arange(len_lat*len_lon*len_depth * len_ts).reshape(len_lat, len_lon, len_depth, len_ts).astype('float64')
    m = obj['thetao'].transpose('latitude','longitude', 'depth_coord', 'time').fillna(-999).astype('float64')

    for lat in range(len_lat):
        for lon in range(len_lon):
            for depth in range(len_depth):
                output[lat,lon,depth] = m[lat,lon,depth,:].load()

    output = output.reshape(len_lat * len_lon * len_depth, len_ts)  
    # mask = output < -999; mask
    # output[mask] = -999
    np.savetxt(f'{fname}.csv', output.astype('float64'), delimiter=',', fmt='%.6f')
#saveIt(obj=s, fname='SODA_temp_Celtic-Sea_30-580m_1958-2007') # obj=input_files

In [52]:
# lat=05, lon=5,depth=5
m[5,5,5].load()

<xarray.DataArray 'thetao' (time: 60)>
array([11.401852 , 10.927594 , 10.693255 , 10.832945 , 11.089084 ,
       11.140119 , 11.144329 , 11.018477 , 11.917805 , 12.182155 ,
       11.372435 , 11.440055 , 11.309369 , 11.111829 , 10.965912 ,
       10.83427  , 10.905318 , 11.655176 , 13.288745 , 14.76556  ,
       14.653581 , 14.136719 , 13.54315  , 12.99344  , 11.943633 ,
       11.663304 , 11.552512 , 11.454192 , 11.846073 , 12.198486 ,
       12.230988 , 12.585258 , 13.073701 , 13.285204 , 13.414214 ,
       13.196177 , 12.406819 , 11.4618025, 11.405766 , 11.485776 ,
       12.20816  , 12.470518 , 12.60076  , 12.989866 , 13.011666 ,
       12.836416 , 13.041643 , 12.566862 , 12.053412 , 11.171542 ,
       10.90778  , 10.902459 , 11.245266 , 11.252755 , 11.290906 ,
       11.852666 , 11.937502 , 11.983492 , 12.534066 , 12.223845 ],
      dtype=float32)
Coordinates:
  * time         (time) datetime64[ns] 2000-01-31 2000-02-29 ... 2004-12-31
    longitude    float64 -9.25
    latitude     float64 49.75
    depth_coord  float32 50.0
Attributes:
    long_name:   TEMPERATURE
    units:       Celsius_scale
    pointwidth:  1.0
    fnname:      flaggt

In [8]:
n_lat=10
n_lon=22
n_depths=21
n_time=60
result = np.empty((n_time,n_lat,n_lon,n_depths))

lat,lon = 0,0
for i, row in enumerate(np.loadtxt('CeltricSea_2000-2004.csv', delimiter=',')):
    d = i % n_depths
    if i % n_depths == 0 and i != 0: lon+=1
    if lon == n_lon:
        lon=0
        lat+=1
    if lat == n_lat: break
    for time in range(n_time): 
        result[time,lat,lon,d] = row[time]    
result[:,5,5,5]

array([11.401852, 10.927594, 10.693255, 10.832945, 11.089084, 11.140119,
       11.144329, 11.018477, 11.917805, 12.182155, 11.372435, 11.440055,
       11.309369, 11.111829, 10.965912, 10.83427 , 10.905318, 11.655176,
       13.288745, 14.76556 , 14.653581, 14.136719, 13.54315 , 12.99344 ,
       11.943633, 11.663304, 11.552512, 11.454192, 11.846073, 12.198486,
       12.230988, 12.585258, 13.073701, 13.285204, 13.414214, 13.196177,
       12.406819, 11.461802, 11.405766, 11.485776, 12.20816 , 12.470518,
       12.60076 , 12.989866, 13.011666, 12.836416, 13.041643, 12.566862,
       12.053412, 11.171542, 10.90778 , 10.902459, 11.245266, 11.252755,
       11.290906, 11.852666, 11.937502, 11.983492, 12.534066, 12.223845])

In [6]:
from pathlib import Path

In [7]:
Path.cwd()

PosixPath('/Users/benjamin/repositories/Growth-Model-Website')

# Compute

In [1]:
from pathlib import Path
import sys, os 
import numpy as np
import pandas as pd
import scipy as sci
import xarray as xr

In [6]:
# !! Celtic SEA

""" Definitions of growth model constants from Butzin and Pörtner, 2016  """

A_R = 8.660        # Rate of uninhibited growth at reference temperature T_R (% d^-1 g^1/b)
B_R = 0.3055       # Value of allometric exponent at reference temperature Tr
THETA_A = 18145    # Arrhenius temperature (K) for uninhibited reaction kinetics = 17871,85°C
THETA_B = 4258     # Arrhenius temperature (K) = 3984,85°C 
THETA_H =  25234   # Arrhenius temperature (K) for inhibited reaction kinetics = 24960,85°C
T_R = 283          # Reference optimum temperature (K) = 9.85°C
T_H = 286          # Temperature for inhibitive processes (K) = 12.85°C
C_AVG =  0.291     # Independent of temperature and weight constant (% d^-1)

""" Definitions of kinetic functions """
Kelvin = 273.15       
def equation2(input_temp):
    temperature_kelvin = input_temp + Kelvin 
    a_numerator = A_R*np.exp(THETA_A/T_R - THETA_A/temperature_kelvin) 
    a_denominator = 1 + np.exp(THETA_H/T_H - THETA_H/temperature_kelvin)
    return a_numerator/a_denominator

def equation3(input_temp):
    """ Arrhenius equation """
    temperature_kelvin = input_temp + Kelvin
    # Calculate b
    return B_R*np.exp(THETA_B/T_R - THETA_B/temperature_kelvin)

experiment_name = 'SODA'
region = 'CelticSea'

out_dir = f'{Path.cwd()}/growth_model_output/{experiment_name}'
if not os.path.exists(out_dir): os.makedirs(out_dir)



# input_files = xr.open_mfdataset('gm/input_data/*.nc', decode_times=False)
# new_time = pd.date_range('2000-01-01', '2005-01-01', freq='M')
# input_files = input_files.assign_coords({'time': new_time})
# input_files = xr.open_mfdataset('gm/input_data/*.nc', decode_times=False) # 2000-2004
input_files = xr.open_dataset('data/SODA_temp_Celtric_Sea_1958-2007_sliced.nc')
new_time = pd.date_range('1958-01-01', '2008-01-01', freq='M')
input_files = input_files.assign_coords({'time': new_time})
#input_files['thetao'].transpose('time','latitude','longitude','depth_coord')[0][0][13][0].load()



# Specification of biological parameters  
# Number of years in one life cycle of an individual 
generation = 10
# The year when the input temeperature dataset starts 
# Should be one year less than starting year in the dataset
initial_year = 1958 # 2000    
# Number of years in the input temperature dataset
years = range(1958, 2008)  	# range(2000, 2005)  	 

# Define geographic boundaries (latitudes, longitudes)
# Default coordinates: North Atlantic coordinates
lat_coords = slice(47,52)
lon_coords = slice(-12,-1)

# Define depth_levels for your growth model output files
depth_levels = slice(0, 600)   # 0-600 meters according to Atlantic cod distribution

# Define latitudes that you will use to save your weight-at-age data to netcdf
lat = np.array(input_files.latitude, dtype='f')
lon = np.array(input_files.longitude, dtype='f')

# Define depth that you will use to save your weight-at-age data to netcdf
depths = np.array(input_files.depth_coord, dtype='f')
# Define dimensionality of coords
N_depths, N_lat, N_lon = len(depths), len(lat), len(lon)

dt = 1


In [7]:
# !! North SEA

# Create output directories if they do not exist

""" Definitions of growth model constants from Butzin and Pörtner, 2016  """

A_R = 8.660        # Rate of uninhibited growth at reference temperature T_R (% d^-1 g^1/b)
B_R = 0.3055       # Value of allometric exponent at reference temperature Tr
THETA_A = 18145    # Arrhenius temperature (K) for uninhibited reaction kinetics = 17871,85°C
THETA_B = 4258     # Arrhenius temperature (K) = 3984,85°C 
THETA_H =  25234   # Arrhenius temperature (K) for inhibited reaction kinetics = 24960,85°C
T_R = 283          # Reference optimum temperature (K) = 9.85°C
T_H = 286          # Temperature for inhibitive processes (K) = 12.85°C
C_AVG =  0.291     # Independent of temperature and weight constant (% d^-1)

""" Definitions of kinetic functions """
Kelvin = 273.15       
def equation2(input_temp):
    temperature_kelvin = input_temp + Kelvin 
    a_numerator = A_R*np.exp(THETA_A/T_R - THETA_A/temperature_kelvin) 
    a_denominator = 1 + np.exp(THETA_H/T_H - THETA_H/temperature_kelvin)
    return a_numerator/a_denominator

def equation3(input_temp):
    """ Arrhenius equation """
    temperature_kelvin = input_temp + Kelvin
    # Calculate b
    return B_R*np.exp(THETA_B/T_R - THETA_B/temperature_kelvin)

experiment_name = 'SODA'
region = 'NorthSea'

out_dir = f'{Path.cwd()}/growth_model_output/{experiment_name}'
if not os.path.exists(out_dir): os.makedirs(out_dir)





# input_files = xr.open_mfdataset('gm/input_data/*.nc', decode_times=False)
# new_time = pd.date_range('2000-01-01', '2005-01-01', freq='M')
# input_files = input_files.assign_coords({'time': new_time})
# input_files = xr.open_mfdataset('gm/input_data/*.nc', decode_times=False) # 2000-2004
input_files = xr.open_dataset('data/SODA_temp_masked_north_atlantic_30-580m_1958-2007_sliced.nc')
new_time = pd.date_range('1958-01-01', '2008-01-01', freq='M')
input_files = input_files.assign_coords({'time': new_time})
#input_files['thetao'].transpose('time','latitude','longitude','depth_coord')[0][0][13][0].load()



# Specification of biological parameters  
# Number of years in one life cycle of an individual 
generation = 10
# The year when the input temeperature dataset starts 
# Should be one year less than starting year in the dataset
initial_year = 1958 # 2000    
# Number of years in the input temperature dataset
years = range(1958, 2008)  	# range(2000, 2005)  	 

# Define geographic boundaries (latitudes, longitudes)
# Default coordinates: North Atlantic coordinates
lat_coords = slice(51, 65)
lon_coords = slice(-3,8)

# Define depth_levels for your growth model output files
depth_levels = slice(30, 600)   # 0-600 meters according to Atlantic cod distribution

# Define latitudes that you will use to save your weight-at-age data to netcdf
lat = np.array(input_files.latitude, dtype='f')
lon = np.array(input_files.longitude, dtype='f')

# Define depth that you will use to save your weight-at-age data to netcdf
depths = np.array(input_files.depth_coord, dtype='f')

dt = 1

# Define dimensionality of coords
N_depths, N_lat, N_lon = len(depths), len(lat), len(lon)


In [8]:
default_dim_labels: tuple = ("depth_coord", "latitude", "longitude")
def save2netcdf(data, var_name:str, directory: str, year, age, dimension_labels: tuple=default_dim_labels):
    if "depth_coord" in dimension_labels:
        coordinates = [('depth_coord', depths), ('latitude', lat), ('longitude', lon)]
    else: coordinates = [('latitude', lat), ('longitude', lon)] 
                
    xr.DataArray(
        data, 
        dims=dimension_labels,
        coords=coordinates, 
        name=var_name
    ).to_netcdf(f'{directory}_{year}_age-{age}.nc')

In [9]:
results = {} # result by year
last_year = initial_year + generation  
while last_year < years[-1]:           
    age, weight = 0, np.ones(shape=(N_depths, N_lat*N_lon), dtype='f')
    growth_rates = np.zeros(shape=(N_depths, N_lat*N_lon), dtype='f')
    age=0

    for every_year in range(initial_year, last_year):
        my_temp = input_files.thetao.sel(
            time=str(every_year), 
            depth_coord = depth_levels, 
            latitude = lat_coords, 
            longitude = lon_coords
        )            

        # Partly vectorize 4D temperature fields  to accelerate the computations
        temp_input_3d = my_temp.values.reshape(12, N_depths, N_lat*N_lon)             
            
        # Set NaN values
        temp_input_3d[np.where(temp_input_3d[:,:,:] <= -998)] = np.nan     

        # Initialize necessary variable fields
        a = np.zeros(shape=(N_depths, N_lat*N_lon), dtype='f')
        b = np.zeros(shape=(N_depths, N_lat*N_lon), dtype='f')

        # mm0 is never used
        if every_year == initial_year: mm0 = 2 
        else: mm0 = 0

        for mon in np.arange(0,12):
            for dd in np.arange(0,30):
                for ilev in np.arange(0, N_depths):  
                    a[ilev, :] = equation2(temp_input_3d[mon, ilev, :])
                    b[ilev, :] = equation3(temp_input_3d[mon, ilev, :]) * (-1.)
                    growth_rates[ilev, :] = 0.01 * ( a[ilev, :] * weight[ilev, :]** b[ilev, :] - C_AVG )  

                    growth_rates[ilev, :] = np.where(growth_rates[ilev, :] < 0,0, growth_rates[ilev, :])
                    weight[ilev, :] = weight[ilev, :] * (1. + dt * growth_rates[ilev, :])

        new_year = int(every_year) + 1
        
        # Reshape data to original shape
        a_3d = a.reshape(N_depths, N_lat, N_lon)
        b_3d = b.reshape(N_depths, N_lat, N_lon)

        growth_rates_3d = growth_rates.reshape(N_depths, N_lat, N_lon)

        # # 3D field with asymptotic weight
        weight_3d = 0.001 * weight.reshape((N_depths, N_lat, N_lon)) 
        
        # # Calculate maximum asymptotic weight at a given location 
        # # ("W*" in Butzin and Pörtner (2016)) 
        weight_max = np.nanmax(weight_3d, axis = 0)

        # optional
        content = {
            'age': age,
            'weight_3d': weight_3d,
            'weight_max': weight_max,
            'a_3d': a_3d,
            'B_3d': b_3d
        }
        if f'{every_year}' not in results.keys(): results[f'{every_year}'] = [content]
        else: results[f'{every_year}'].append(content)
        
        # Save netcdf
        # for i, (data, variable_name) in enumerate(zip(
        #     [a_3d, b_3d, growth_rates_3d, weight_3d, weight_max],
        #     ['a_3d', 'b_3d', 'growth_rates_3d', 'weight_3d', 'weight_max'],
        # )):
        #     save2netcdf(
        #         data=data, 
        #         var_name=variable_name, 
        #         directory=f'{out_dir}/init_initial_year_transient' , 
        #         year=every_year, 
        #         age=age, 
        #         dimension_labels=default_dim_labels if i < 4 else  ('latitude', 'longitude')
        #     )
        age += 1
            
    initial_year = initial_year + 1
    last_year = initial_year + generation 


/var/folders/j1/k2clnjz530q64v44gw4srhpw0000gn/T/ipykernel_66851/1680215221.py:53: RuntimeWarning: All-NaN slice encountered
  weight_max = np.nanmax(weight_3d, axis = 0)


In [53]:
for item in results['2000']:
    print(item['age'])

0


# Precompute to save

In [ ]:
weight_by_age_quantile = [[] for _ in range(generation)]

for year in results.keys():
    for i in results[f'{year}']:
        age = i['age']
        [weight_by_age_quantile[age].append(x) for x in i['weight_3d'].ravel()]

NameError: name 'generation' is not defined

In [17]:
quantiles = [[] for _ in range(len([.25,.5,.75]))]

for i in range(generation):
    quantiles[0].append(np.nanquantile(weight_by_age_quantile[i],.25))
    quantiles[1].append(np.nanquantile(weight_by_age_quantile[i],.5))
    quantiles[2].append(np.nanquantile(weight_by_age_quantile[i],.75))

In [18]:
np.savetxt(
    f'North-Sea_1958-2007_gen-10_quantiles_precomputed.csv', 
    np.array(quantiles).astype('float64'), delimiter=',', fmt='%.6f')

In [19]:
np.array(quantiles)

array([[ 0.21038864,  1.1640366 ,  2.88889539,  5.05817902,  7.34373844,
         9.51292801, 11.46276188, 13.12087679, 14.50960994, 15.65978527],
       [ 0.2605017 ,  1.38585854,  3.3426764 ,  5.75219512,  8.24842024,
        10.5780158 , 12.62991762, 14.35769415, 15.77229786, 16.9244318 ],
       [ 0.31259235,  1.60936734,  3.80526805,  6.44904673,  9.12173629,
        11.57055759, 13.69589877, 15.45860839, 16.89059448, 18.03893518]])

In [ ]:
# 

# Save precomputed boxplotdata / mean weight per age and year
```bash
year,1,2,3,4,5,6,7,8,9,10
1971,xx,xx...
1972,xx,xx...
1973,xx,xx...
```

In [12]:
d={};d['year'] = []; d = { gen: [] for gen in range(generation) }
df = pd.DataFrame(data=d); df

,0,1,2,3,4,5,6,7,8,9


In [45]:
new_row = { 
    'year': '1997', 
    0:1278364,
    1:127381,
    2:126973,
    3:189234,
    4:127381,
    5:126973,
    6:189234,
    7:127381,
    8:126973,
    9:189234,
    10:189234
}

In [50]:
df.append(new_row, ignore_index=True)

,0,1,2,3,4,5,6,7,8,9,10,year
0,1278364.0,127381.0,126973.0,189234.0,127381.0,126973.0,189234.0,127381.0,126973.0,189234.0,189234.0,1997


In [29]:
d={}; d['year'] = []; d = { gen: [] for gen in range(generation) }
df = pd.DataFrame(data=d); df

for year in results.keys():
    new_row = { 
        'year': f'{year}', 
        0: None,
        1: None,
        2: None,
        3: None,
        4: None,
        5: None,
        6: None,
        7: None,
        8: None,
        9: None
    }
    for i in results[f'{year}']:
        new_row[int(i['age'])] = np.nanmean(i['weight_3d'])
    df = df.append(new_row, ignore_index=True)

In [35]:
df[['year',0, 1 , 2, 3, 4, 5, 6, 7, 8, 9]].to_csv('Celtic-Sea-Boxplot_data.csv', index=False)